In [3]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from skimage.measure import compare_psnr,compare_mse
from scipy import ndimage, misc
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras.layers import *
from keras.models import Model
import re
from keras import regularizers
from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import mean_squared_error
import scipy.io as sio
import pickle
from random import randint
from utilities import *
from PIL import Image
%matplotlib inline


In [4]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append(img)
    return np.array(images)
        

In [5]:
def resize_images(src_path,dest_path,sizeX,sizeY):
    images = []
    width,height = sizeX,sizeY
    for filename in os.listdir(src_path):
        img = cv2.imread(src_path + filename)
        if img is not None:
            images.append(img)
    cnt =1
    for img in images:
        im2 = cv2.resize(img,(width, height),interpolation = cv2.INTER_CUBIC)
        
        
        ext = ".png"
        cv2.imwrite(dest_path + "image" +  str(cnt) + ext,im2)
        cnt+=1

In [5]:
raw_images = read_images_from_folder('./data/oct_gt/')
raw_images = np.resize(raw_images,(raw_images.shape[0],448,900,1))

In [6]:
#Adding noise to images
def add_noise(images):
    
    noisy_set = []
    for image in images:
        print len(noisy_set)
        for shape in xrange(2,9,1):
            for scale in xrange(2,9,1):
                noise = np.random.gamma(shape,scale,image.shape)
                noisy_set.append((image,np.clip(noise + image,0.,255.)))
    shuffle(noisy_set)
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def expand_pair(noisy_set):   
    ground_truth=[]
    noisy_images = []
    for i in range(noisy_set.shape[0]):
        ground_truth.append(noisy_set[i][0].reshape((noisy_set[i][0].shape[0],noisy_set[i][0].shape[1],1)))
        #print( str(noisy_set[i][0].shape[0]) +" "+ str(noisy_set[i][0].shape[1]))
        noisy_images.append(noisy_set[i][1].reshape((noisy_set[i][1].shape[0],noisy_set[i][1].shape[1],1)))
    return np.array(ground_truth), np.array(noisy_images)

In [7]:
noisy_set = add_noise(raw_images)
for i in range(noisy_set.shape[0]):
    cv2.imwrite("./data/oct_noisy/"+str(i)+".tif",noisy_set[i][1])    

0
49
98
147
196
245
294
343
392
441
490
539
588
637
686
735
784


In [8]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = expand_pair(noisy_set)
print (ground_truth.shape, noisy_images.shape, noisy_set.shape)

((833, 448, 900, 1), (833, 448, 900, 1), (833, 2, 448, 900, 1))


In [9]:
print np.std(ground_truth/255.)

0.125008434593


In [10]:
def resize_images(images,size):
    resized_images = []
    
    for img in images:
        im2 = cv2.resize(img,size,interpolation = cv2.INTER_CUBIC)
        resized_images.append(im2)
    
    resized_images = np.array(resized_images)
    resized_images = np.reshape(resized_images,(resized_images.shape[0],resized_images.shape[1],resized_images.shape[2],1))
    
    return resized_images

ground_truth_rs = resize_images(ground_truth,(256,128))
noisy_images_rs = resize_images(noisy_images,(256,128))

In [11]:
noisy_images_rs.shape

(833, 128, 256, 1)

In [12]:
for i in range(noisy_images_rs.shape[0]):
    cv2.imwrite("./data/oct_noisy_resized/"+str(i)+".tif",noisy_images_rs[i])  
    
for i in range(ground_truth_rs.shape[0]):
    cv2.imwrite("./data/oct_gt_resized/"+str(i)+".tif",ground_truth_rs[i])    

In [6]:
ground_truth_rs = read_images_from_folder('./data/oct_gt_resized/')
ground_truth_rs = np.resize(ground_truth_rs,(ground_truth_rs.shape[0],ground_truth_rs.shape[1],ground_truth_rs.shape[2],1))

noisy_images_rs = read_images_from_folder('./data/oct_noisy_resized/')
noisy_images_rs = np.resize(noisy_images_rs,(noisy_images_rs.shape[0],noisy_images_rs.shape[1],noisy_images_rs.shape[2],1))

In [7]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth_rs.shape[0]*0.8)
x_train = ground_truth_rs[0:train_size]/255.
x_train_noisy = noisy_images_rs[0:train_size]/255.
x_test = ground_truth_rs[train_size:]/255.
x_test_noisy = noisy_images_rs[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(666, 128, 256, 1)
(167, 128, 256, 1)


In [6]:
#lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
#csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
#early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [ ]:
x_train_noisy.shape

(666, 128, 256, 1)

In [3]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=((None,None,1)))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        
        
        autoencoder.compile(optimizer='Adam', loss='mean_squared_error',metrics = ['accuracy','mean_squared_error'])

    print (autoencoder.summary())
    return autoencoder

In [4]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
__________

In [78]:
plot_model(autoencoder, to_file='model.png',show_shapes='True',show_layer_names='True')

In [5]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

NameError: name 'x_train_noisy' is not defined

In [8]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

def _tf_fspecial_gauss(size, sigma):
    """Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x_data, y_data = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]

    x_data = np.expand_dims(x_data, axis=-1)
    x_data = np.expand_dims(x_data, axis=-1)

    y_data = np.expand_dims(y_data, axis=-1)
    y_data = np.expand_dims(y_data, axis=-1)

    x = tf.constant(x_data, dtype=tf.float32)
    y = tf.constant(y_data, dtype=tf.float32)

    g = tf.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g / tf.reduce_sum(g)

def tf_ssim(img1, img2, cs_map=False, mean_metric=True, size=11, sigma=1.5):
    window = _tf_fspecial_gauss(size, sigma) # window shape [size, size]
    K1 = 0.01
    K2 = 0.03
    L = 1  # depth of image (255 in case the image has a differnt scale)
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = tf.nn.conv2d(img1, window, strides=[1,1,1,1], padding='VALID')
    mu2 = tf.nn.conv2d(img2, window, strides=[1,1,1,1],padding='VALID')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = tf.nn.conv2d(img1*img1, window, strides=[1,1,1,1],padding='VALID') - mu1_sq
    sigma2_sq = tf.nn.conv2d(img2*img2, window, strides=[1,1,1,1],padding='VALID') - mu2_sq
    sigma12 = tf.nn.conv2d(img1*img2, window, strides=[1,1,1,1],padding='VALID') - mu1_mu2
    if cs_map:
        value = (((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2)),
                (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2))
    else:
        value = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2))

    if mean_metric:
        value = tf.reduce_mean(value)
    return value

def tf_ms_ssim(img1, img2, mean_metric=True, level=5):
    weight = tf.constant([0.0448, 0.2856, 0.3001, 0.2363, 0.1333], dtype=tf.float32)
    mssim = []
    mcs = []
    for l in range(level):
        ssim_map, cs_map = tf_ssim(img1, img2, cs_map=True, mean_metric=False)
        mssim.append(tf.reduce_mean(ssim_map))
        mcs.append(tf.reduce_mean(cs_map))
        filtered_im1 = tf.nn.avg_pool(img1, [1,2,2,1], [1,2,2,1], padding='SAME')
        filtered_im2 = tf.nn.avg_pool(img2, [1,2,2,1], [1,2,2,1], padding='SAME')
        img1 = filtered_im1
        img2 = filtered_im2

    # list to tensor of dim D+1
    mssim = tf.stack(mssim, axis=0)
    mcs = tf.stack(mcs, axis=0)

    value = (tf.reduce_prod(mcs[0:level-1]**weight[0:level-1])*
                            (mssim[level-1]**weight[level-1]))

    if mean_metric:
        value = tf.reduce_mean(value)
    return value

def dssim_objective(y_pred,y_true):
    ssim = tf_ssim(y_pred,y_true)
    return (1 - ssim)/2.
    

In [56]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    fractionG = Lambda(lambda x, factor : factor * x, arguments = {'factor': gatePercentageFactor})(inputLayer)
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,1), name = "Image_input")  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        fg1,c1 = get_gated_connections(0.1,x1)
        
        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(fg3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(fg4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model(input_img,layers)
        sym_autoencoder.compile(optimizer='adam', loss = dssim_objective, metrics = ['accuracy',dssim_objective])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [57]:
sym_autoencoder = get_cnn_dsc_architecture()

Model created
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image_input (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, None, None, 6 1088        Image_input[0][0]                
__________________________________________________________________________________________________
lambda_89 (Lambda)              (None, None, None, 6 0           conv2d_61[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_47 (MaxPooling2D) (None, None, None, 6 0           lambda_89[0][0]                  
_______________________________________________________________________________________________

In [58]:
model_checkpoint1 = ModelCheckpoint('./models/gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [59]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs = 1,
                batch_size = 8,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 666 samples, validate on 167 samples
Epoch 1/1
192/666 [=======>......................] - ETA: 11:53 - loss: 0.1331 - acc: 0.0000e+00 - dssim_objective: 0.1331

KeyboardInterrupt: 

In [51]:
#Testing with gated model
def non_ref_img_denoise_metric(noisy_img,denoised_img):
    mni = noisy_img - denoised_img
    ssim_n = np.array(compare_ssim(noisy_img,mni,multichannel=True,full=True)[1])
    ssim_p = np.array(compare_ssim(noisy_img,denoised_img,multichannel=True,full=True)[1])
    
    return pearsonr(ssim_n.flatten(),ssim_p.flatten())


In [11]:
sym_autoencoder = load_model('./models/gated_oct_epoch_400_batch_8_sgd/gated_cnn_autoencoder_oct.hdf5')


In [12]:
def read_images_from_folder_with_names(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((filename,img))
    return images

In [13]:
test_image_set = read_images_from_folder_with_names('./test_data/cropped_noisy/')

In [18]:
gt = cv2.imread('test_data/cropped5.tif',0)
gt = np.array(gt)
gt = cv2.resize(gt,(896,288),interpolation = cv2.INTER_CUBIC)
gt =[gt]
gt = np.array(gt,dtype=np.float32)
gt = gt/255.
gt = gt.reshape((1,288,896,1))
gt.shape

(1, 288, 896, 1)

In [59]:
for idx,test_image in enumerate(test_image_set):
    test_image_norm = cv2.resize(test_image[1],(896,288),interpolation = cv2.INTER_CUBIC)
    test_image_norm = [test_image_norm]
    test_image_norm = np.array(test_image_norm,dtype=np.float32)
    test_image_norm = test_image_norm/255.

    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    out_image = sym_autoencoder.predict(test_image_norm,verbose=1)
    #print (out_image[0].shape,gt.shape)
    psnr = compare_psnr(gt[0],out_image[0])
    ssim = compare_ssim(gt[0],out_image[0],multichannel=True)
    
    print test_image_norm[0].shape
    nlm = cv2.fastNlMeansDenoisingColored(test_image_norm[0].astype(int),None,10,10,7,21)
    non_ref_metric = non_ref_img_denoise_metric(test_image_norm[0],out_image[0])
    nlm_non_ref_metric = non_ref_img_denoise_metric(test_image_norm[0],nlm)
    
    print test_image[0][:-3]
    print psnr,ssim,non_ref_metric
   
    #cv2.imwrite('./test_data/cropped_denoised_tanh/'+test_image[0][:-3] + '_'+str(round(psnr,4))+'_'+str(round(ssim,5))+'.tif',out_image[0]*255.)
    

1/1 [==============================] - 4s 4s/step
(288, 896, 1)


error: /feedstock_root/build_artefacts/opencv_1506439658817/work/opencv-3.3.0/modules/photo/src/denoising.cpp:183: error: (-5) Type of input image should be CV_8UC3 or CV_8UC4! in function fastNlMeansDenoisingColored


In [ ]:
#Testing with simple model

In [ ]:
model = load_model("./models/oct_epoch_500_batch_20/simple_cnn_oct.hdf5")

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
average_pooling2d_1 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_2 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
__________

In [7]:
#Adding Gamma noise
test_image = cv2.imread('./test_data/cropped5.tif',0)
print test_image.shape
#test_image = cv2.resize(test_image,(900,448),interpolation = cv2.INTER_CUBIC)
#print test_image.shape
gamma_param = [(3,2),(4,3),(4,4),(6,6),(8,7)]
for param in gamma_param:
    shape = param[0]
    scale = param[1]
    noise = np.random.gamma(shape,scale,test_image.shape)
    noisy_img = np.clip(noise + test_image,0.,255.)
    cv2.imwrite('./test_data/cropped_noisy/oct_shape_'+str(shape)+'_scale_'+str(scale)+'.tif',noisy_img)

(280, 900)


In [13]:
test_image_set = read_images_from_folder_with_names('./test_data/cropped_noisy/')

In [14]:
test_image_set

[('oct_shape_4_scale_3.tif', array([[73, 85, 81, ..., 87, 78, 82],
         [78, 89, 82, ..., 76, 75, 82],
         [90, 94, 91, ..., 84, 88, 91],
         ..., 
         [81, 81, 90, ..., 80, 85, 85],
         [77, 68, 84, ..., 85, 91, 88],
         [96, 70, 96, ..., 76, 77, 86]], dtype=uint8)),
 ('oct_shape_6_scale_6.tif', array([[114,  97, 146, ...,  92, 104, 101],
         [ 95, 104, 106, ..., 111,  80, 121],
         [114, 122, 103, ...,  91, 103, 129],
         ..., 
         [106, 104,  83, ..., 127, 112,  98],
         [127, 103,  90, ..., 102, 142, 101],
         [ 87, 112, 114, ..., 137, 121, 110]], dtype=uint8)),
 ('oct_shape_3_scale_2.tif', array([[73, 73, 82, ..., 79, 75, 79],
         [75, 77, 77, ..., 72, 71, 74],
         [72, 81, 83, ..., 75, 78, 87],
         ..., 
         [76, 70, 71, ..., 77, 81, 81],
         [78, 67, 84, ..., 79, 79, 71],
         [74, 70, 71, ..., 72, 75, 85]], dtype=uint8)),
 ('oct_shape_8_scale_7.tif', array([[122, 113, 126, ..., 113, 135, 127

In [15]:
for idx,test_image in enumerate(test_image_set):
    test_image_norm = cv2.resize(test_image[1],(792,792),interpolation = cv2.INTER_CUBIC)

    test_image_norm = [test_image_norm]
    test_image_norm = np.array(test_image_norm,dtype=np.float64)
    test_image_norm = test_image_norm/255.
    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    
    out_image = model.predict(test_image_norm,verbose=1)
    out_image = out_image.astype(np.float64)
    print test_image_norm[0].shape,out_image[0].shape
    psnr = compare_psnr(test_image_norm[0]*255.,out_image[0]*255.,data_range=256)
    ssim = compare_ssim(test_image_norm[0],out_image[0],multichannel=True)
    print psnr,ssim
    print test_image[0][:-3]
    #cv2.imwrite('./test_data/cropped_denoised/'+test_image[0][:-3] + '_'+str(round(psnr,4))+'_'+str(round(ssim,5))+'.tif',out_image[0]*255.)
    print test_image_norm.shape


NameError: name 'model' is not defined

In [47]:
a=[[1,2],[3,4]]
a= np.array(a)

In [50]:
a.flatten()

array([1, 2, 3, 4])